<a href="https://colab.research.google.com/github/t1seo/AIFFEL/blob/master/EXPLORATION/01/%EB%AF%B8%EB%8B%88%20%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8%20-%20%EA%B0%80%EC%9C%84%EB%B0%94%EC%9C%84%EB%B3%B4%20%EB%B6%84%EB%A5%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 미니 프로젝트 : 가위바위보 분류기를 만들자

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 1. 데이터 준비

### 1-1. 데이터 만들기
노트북 카메라를 이용하여 가위, 바위, 보 이미지 각 100장을 만들어본다. 구글의 teachable machine 사이트에서 쉽게 데이터를 만들 수 있다.

[https://teachablemachine.withgoogle.com/](https://teachablemachine.withgoogle.com/)

- 여러 각도에서 찍기
- 여러 크기로 찍기
- 여러 명이 찍기

좋은 데이터가 좋은 결과를 낳는다.

teachable machine을 통해서 이미지를 저장하면 이미지는 224x224 크기로 되어 있다.

### 1-2. 데이터 불러오기 + Resize 하기
가위, 바위, 보 이미지를 28x28로 만들어야 한다. 이를 위해 `PIL` 라이브러리를 사용한다.

In [ ]:
from PIL import Image
import os, glob
# !nvidia-smi

In [ ]:
import os

IMG_SIZE = 28 # 이미지 사이즈
total_image = 0 # 전체 이미지 개수

# 이미지를 리사이즈 해주는 함수
def resize_images(img_path):
    images=glob.glob(img_path + "/*.jpg")
    print(f"{len(images)} images will be resized.")
    
    global total_image 
    total_image += len(images)
    
    # resize all images to 28x28
    target_size=(IMG_SIZE, IMG_SIZE) # 28x28 size
    for img in images:
        old_img=Image.open(img)
        new_img=old_img.resize(target_size, Image.ANTIALIAS)
        new_img.save(img, "JPEG")
        
    print(f"{len(images)} images have been resized.\n")

In [ ]:
# local
# data_dir = os.path.dirname(os.path.abspath('__file__')) # 현재 위치
# data_dir += r'/data/' # 데이터 폴더 위치
# print(data_dir)

# train_path = data_dir + r'train/'
# scissor_train_path = train_path + r'scissors/'
# rock_train_path = train_path + r'rock/'
# paper_train_path = train_path + r'paper/'

# train_paths = [scissor_train_path, rock_train_path, paper_train_path]
# print(train_paths)

In [ ]:
## Colab
paper_dir = "/content/drive/MyDrive/data/train/paper"
scissor_dir = "/content/drive/MyDrive/data/train/scissors"
rock_dir = "/content/drive/MyDrive/data/train/rock"

img_dir = [scissor_dir, rock_dir, paper_dir]

In [ ]:
# scissor_dir = os.getenv("HOME") + "/aiffel/rock_scissor_paper/scissor"
# rock_dir = os.getenv("HOME") + "/aiffel/rock_scissor_paper/rock"
# paper_dir = os.getenv("HOME") + "/aiffel/rock_scissor_paper/paper"

# # TODO: 폴더 존재하는지 체크해주는 부분 추가

# img_dir = [scissor_dir, rock_dir, paper_dir]

In [ ]:
from tqdm import tqdm

for image in tqdm(img_dir):
    resize_images(image)
    
print(f"총 {total_image} 이미지 resize 완료!")

In [ ]:
# 총 이미지 개수
print(total_image)

### 1-3. 가위, 바위, 보 데이터를 읽는 `load_data()` 함수 만들기

`load_data()` 함수는 입력으로 이미지가 있는 폴더 위치를 받는다. 여기서는 `rock_scissor_paper` 폴더 위치를 적어주면 된다.

In [ ]:
import numpy as np

def load_data(img_path, number_of_data=300): 
    """    
    parameters:
        img_path: The directory path of the rock_scissor_paper
        number_of_data: The total nubmer of all images. Default value is 300.
    return:
        image data, label data
    """
    # scissors: 0, rock: 1, paper: 2
    img_size = IMG_SIZE
    color = 3
    
    # image data
    imgs = np.zeros(number_of_data * img_size * img_size * color, 
                    dtype=np.int32).reshape(number_of_data, img_size, img_size, color)
    labels = np.zeros(number_of_data, dtype=np.int32)
    
    idx = 0
    for file in glob.iglob(img_path + '/scissors/*.jpg'):
        img = np.array(Image.open(file), dtype=np.int32)
        imgs[idx, :, :, :] = img # 데이터 영역에 이미지 행렬을 복사
        labels[idx] = 0 # 가위 : 0
        idx = idx + 1
        
    for file in glob.iglob(img_path + '/rock/*.jpg'):
        img = np.array(Image.open(file), dtype=np.int32)
        imgs[idx, :, :, :] = img # 데이터 영역에 이미지 행렬을 복사
        labels[idx] = 1 # 바위 : 1
        idx = idx + 1
    
    for file in glob.iglob(img_path + '/paper/*.jpg'):
        img = np.array(Image.open(file), dtype=np.int32)
        imgs[idx, :, :, :] = img # 데이터 영역에 이미지 행렬을 복사
        labels[idx] = 2 # 보 : 2
        idx = idx + 1
        
    print(f"총 이미지 개수는 {idx} 입니다.")
    return imgs, labels

In [ ]:
# train_data_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper"
train_data_path = "/content/drive/MyDrive/data/train"

(X, y) = load_data(train_data_path, number_of_data=3207)

### 1-4. train, test data 나누기

In [ ]:
# train data와 test data 나누어 주기
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))

print("X_test shape: {}".format(X_test.shape))
print("y_test shape: {}".format(y_test.shape))

In [ ]:
# 정규화
X_train = X_train / 255.0   # 입력을 0~1 사이의 값으로 정규화

In [ ]:
# 이미지 확인 (가위:0, 바위:1, 보:2)
import matplotlib.pyplot as plt

plt.imshow(X_train[1000])
print('라벨: ', y_train[1000])

## 2. 딥러닝 네트워크 설계하기

In [ ]:
import tensorflow as tf
from tensorflow import keras

# mini VGGNet
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(input_shape=(IMG_SIZE, IMG_SIZE, 3), kernel_size=(3, 3), 
                           filters=32, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=128, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=256, padding='valid', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=3, activation='softmax')
])

In [ ]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Conv2D(input_shape=(IMG_SIZE, IMG_SIZE, 3), kernel_size=(3, 3), 
#                            filters=16, padding='same', activation='relu'),
#     tf.keras.layers.Conv2D(kernel_size=(3,3), filters=32, padding='same', activation='relu'),
#     tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
#     tf.keras.layers.Dropout(rate=0.5),
#     tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=64, padding='same', activation='relu'),
#     tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=128, padding='valid', activation='relu'),
#     tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
#     tf.keras.layers.Dropout(rate=0.5),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(units=256, activation='relu'),
#     tf.keras.layers.Dropout(rate=0.5),
#     tf.keras.layers.Dense(units=128, activation='relu'),
#     tf.keras.layers.Dropout(rate=0.5),
#     tf.keras.layers.Dense(units=3, activation='softmax') # 10->3
# ])

In [ ]:
model.summary()

## 3. 딥러닝 네트워크 학습시키기

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=25, validation_split=0.25)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], marker='.', c='red', label='Train-set Loss')
plt.plot(history.history['val_loss'], marker='.', c='blue', label='Validation-set Loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], 'g--', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
plt.ylim(0, 1.2)
plt.legend()

plt.show()

## 4. test 데이터로 테스트

In [ ]:
X_test = X_test / 255.0 # 

test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)

print(f"test_loss: {test_loss}")
print(f"test_accuracy: {test_accuracy}")

In [ ]:
predicted_result = model.predict(X_test) # model이 추론한 확률값
predicted_labels = np.argmax(predicted_result, axis=1) # 확률의 최대값이 예측하는 숫자를 뜻한다

idx = 324
print(f"models.predict() 결과: {predicted_result[idx]}\n")
print(f"model이 추론한 가장 가능성이 높은 결과 : ", predicted_labels[idx])
print(f"실제 데이터의 라벨 : ", y_test[idx])

In [ ]:
# 실제 이미지 출력
plt.imshow(X_test[idx], cmap=plt.cm.binary)
plt.show()

## 5. 완전히 새로운 데이터로 테스트

In [ ]:
# scissor_test = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/scissor"
# rock_test = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/rock"
# paper_test = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test/paper"
# test_path = paper_test_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test"

# colab
paper_test = "/content/drive/MyDrive/data/test/paper"
scissor_test = "/content/drive/MyDrive/data/test/scissors"
rock_test = "/content/drive/MyDrive/data/test/rock"
test_path = "/content/drive/MyDrive/data/test"

test_paths = [scissor_test, rock_test, paper_test]

# 새로운 데이터 리사이즈
for path in test_paths:
    resize_images(path) 

(X_new, y_new)=load_data(test_path)
X_new_norm = X_new / 255.0



test_loss, test_accuracy = model.evaluate(X_new_norm, y_new, verbose=2)

print(f"test_loss: {test_loss}")
print(f"test_accuracy: {test_accuracy}")

## 6. 더 좋은 네트워크 만들어보기
- [x] 더 많은 데이터 추가하기
- [x] Validation Set 만들기
- [x] 하이퍼파라미터 변경
- [x] 딥러닝 모델 바꾸기

### 6-1. 이미지 보강

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

image_generator = ImageDataGenerator(
            rotation_range=10,
            zoom_range=0.10,
            shear_range=0.5,
            width_shift_range=0.10,
            height_shift_range=0.10,
            horizontal_flip=True,
            vertical_flip=False)

augment_size = 30000

randidx = np.random.randint(X_train.shape[0], size=augment_size)
x_augmented = X_train[randidx].copy()
y_augmented = y_train[randidx].copy()
x_augmented = image_generator.flow(x_augmented, np.zeros(augment_size),
                                   batch_size=augment_size, shuffle=False).next()[0]

In [ ]:
X_train = np.concatenate((X_train, x_augmented))
y_train = np.concatenate((y_train, y_augmented))

print(X_train.shape)

### 6-2. 다시 학습 및 테스트

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(input_shape=(IMG_SIZE, IMG_SIZE, 3), kernel_size=(3, 3), 
                           filters=32, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=128, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(kernel_size=(3, 3), filters=256, padding='valid', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=3, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=25, validation_split=0.25)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], marker='.', c='red', label='Train-set Loss')
plt.plot(history.history['val_loss'], marker='.', c='blue', label='Validation-set Loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], 'g--', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
plt.ylim(0, 1.2)
plt.legend()

plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)

print(f"test_loss: {test_loss}")
print(f"test_accuracy: {test_accuracy}")

In [ ]:
paper_test = "/content/drive/MyDrive/data/test/paper"
scissor_test = "/content/drive/MyDrive/data/test/scissors"
rock_test = "/content/drive/MyDrive/data/test/rock"
test_path = "/content/drive/MyDrive/data/test"

test_paths = [scissor_test, rock_test, paper_test]

# 새로운 데이터 리사이즈
for path in test_paths:
    resize_images(path) 

(X_new, y_new)=load_data(test_path)
X_new_norm = X_new / 255.0



test_loss, test_accuracy = model.evaluate(X_new_norm, y_new, verbose=2)

print(f"test_loss: {test_loss}")
print(f"test_accuracy: {test_accuracy}")

## 7. 결론(회고)

- 책을 보고 공부하면서 많이 배웠는데 다음과 같은 내용들을 배웠다.
    - Sequential model 만드는 방법
    - 모델의 각 layer의 의미
    - compile 옵션과 그 의미
    - history와 history 그래프 그리기
    - 이미지 보강하여 데이터 증폭 시키기
  
  
- 평가 지표를 맞추기 위해 다음과 같은 시도를 했다.
    - 더 많은 데이터 추가하기: 처음에 내가 찍은 사진 데이터 300장으로만 테스트 했는데 정확도 15%가 나왔었다. 이 후 사진을 더 추가하여 3000여장으로 늘렸다. 그 결과 정확도가 40% 정도 되었다.
    - train, test 세트 나누기: 처음에는 train, test 셋으로 나누지 않고, 전체 데이터를 학습 시킨 후 완전 새로운 데이터로 테스트를 했었다. 이 방법 대신 train_test_split으로 train, test 셋을 나누어주었다. 그 결과 정확도가 70~80% 정도가 되었다. 완전 새로운 데이터보다는 정확도가 더 높게 나오게 된 것이다.
    - Validation Set 만들기: train, test set 뿐만 아니라 validation set도 추가하였다. 이전에는 scikit-learn의 train_test_split을 두 번 사용해서 나눴었는데, tensorflow의 fit에 옵션을 validation_split 넣어주면서 간단히 validation set이 나누어 주었다.
    - history 부분 추가: history를 이용해 그래프를 그림으로써 나의 모델이 무엇이 부족한지 한 눈에 보이도록 만들었다.
    - 딥러닝 모델 바꾸기: 책을 보면서 컨벌루션 레이어, 풀링 레이어, 드롭 아웃 레이어 등을 추가 하였다. 
    - 하이퍼파라미터 변경: 하이퍼파라미터 부분은 손을 대면 댈수록 결과가 나빠져서 되도록 그대로 두었다. 
    - 이미지 데이터 증폭: 이 부분이 신기했다. 이미지들을 약간씩 수정해 데이터를 증폭시켜 이미지 데이터의 부족함을 극복하는 것이다. 그 결과 테스트 정확도가 100%가 되었다.
- Tensorflow를 사실상 처음 사용해봤고 딥러닝에 대한 지식 특히 CNN에 대한 지식이 거의 없어서 많이 힘들었다. 이번 프로젝트에서 만든 모델의 각 부분이 무엇을 의미하는지는 알겠는데 왜 이렇게 레이어들을 쌓았고, 하이퍼파라미터 값은 왜 이 값으로 넣었는지는 아직 잘 모르겠다. CNN, Tensorflow에 대해 더 공부해야 한다.